In [1]:
import numpy as np
import dxchange
import tomopy
import tomocupy

from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
import tkinter as tk
from tkinter.filedialog import askopenfile

/home/beams/TOMO/conda/miniforge3/envs/tomopyui/lib/python3.9/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [2]:
# Utility Functions

In [3]:
def read_filename():
    window = tk.Tk()
    window.wm_attributes('-topmost', 1)
    window.withdraw()

    file = askopenfile(mode ='r', filetypes =[('Python Files', '*.h5')])
    window.destroy()
    return file.name

# Load data

In [4]:
# !wget -nc https://anl.box.com/shared/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz
# file_name = read_filename()
file_name = '/data/2022-12/Luxi_173.h5'
dark_file_name = file_name
flat_file_name = file_name

# Read data, dark and flat fields, projection angles, metadata

In [5]:
# data = np.random.rand(100,256,256) #3d-array with 100 frames 256x256
data, flat, dark, theta = dxchange.read_aps_tomoscan_hdf5(file_name)#, sino=(100, 400))
tt, meta_dict = dxchange.read_hdf_meta(file_name)
# print(meta_dict)

In [6]:
data.shape 

(1800, 852, 2800)

# Plot projections

In [7]:
def plot_projections(angle=1):
    plt.imshow(data[angle,:],cmap='gray')
interact(plot_projections, angle = widgets.IntSlider(name='Projection', value=data.shape[0]//2,
                                               min=0,
                                               max=data.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=900, description='angle', max=1799), Output()), _dom_classes=('widget-in…

<function __main__.plot_projections(angle=1)>

In [12]:
from types import SimpleNamespace

params_dict = {}
for section in config.RECON_STEPS_PARAMS:
    # print(section)
    for key, value in config.SECTIONS[section].items():
        # print('%s = %s ' % (key, value['default']))
        key = key.replace('-', '_')
        params_dict[key] = value['default']

# print(params_dict)

args = SimpleNamespace(**params_dict)
print(args)



args1 = SimpleNamespace(rotation_axis_auto           = 'auto', 
                       file_name                    = file_name,
                       dark_file_name               = file_name, 
                       flat_file_name               = file_name, 
                       rotation_axis_method         = 'sift',  
                       binning                      = 0,
                       start_row                    = 0,
                       end_row                      = -1,
                       start_proj                   = 0,
                       end_proj                     = -1,
                       start_column                 = 0,
                       end_column                   = -1,
                       nsino_per_chunk              = 8,
                       nproj_per_chunk              = 8,
                       rotation_axis                = -1,
                       file_type                    = 'standard',
                       dtype                        = 'float16',
                       blocked_views                = 'none',
                       nsino                        = '0.5',
                       reconstruction_type          = 'try',
                       center_search_width          =  50,
                       center_search_step           =  0.5,
                       lamino_search_width          =  5.0,
                       lamino_search_step           =  0.25,
                       lamino_angle                 =  0,
                       lamino_start_row             =  0,
                       lamino_end_row               =  -1,
                       reconstruction_algorithm     = 'fourierrec',
                       fbp_filter                   = 'shepp',
                       rotation_axis_pairs          = '[0,0]',
                       find_center_start_row        = 0,
                       find_center_end_row          = -1,
                       flat_linear                  = 'False',
                       rotation_axis_sift_threshold = 0.5,
                       out_path_name                = '/data/tmpfdc/',
                       clear_folder                 = True,
                       max_write_threads            = 8,
                       max_read_threads             = 4,
                       dezinger                     = 0,
                       remove_stripe_method         = 'vo',
                       retrieve_phase_method        = None,
                       pixel_size                   = meta_dict['measurement_instrument_detection_system_objective_resolution'][0] * 1e-4,
                       propagation_distance         = meta_dict['measurement_instrument_monochromator_energy'][0]/10,
                       energy                       = meta_dict['measurement_instrument_detector_motor_stack_setup_z'][0],
                       retrieve_phase_alpha         = 0.2,
                       rotate_proj_angle            = 0,
                       rotate_proj_order            = 1,
                       minus_log                    = True
                      )

clrotthandle = tomocupy.FindCenter(args)
args.rotation_axis = clrotthandle.find_center()*2**args.binning
print(f'set rotaion  axis {args.rotation_axis}')


Using dark fields from /data/2022-12/Luxi_173.h5
Using flat fields from /data/2022-12/Luxi_173.h5
Crop data to the power of 2 sizes to work with 16bit precision, output size in x dimension 2048


set rotaion  axis 1407.6629638671875


In [13]:
clpthandle = tomocupy.GPURec(args)
clpthandle.recon_try()

Using dark fields from /data/2022-12/Luxi_173.h5
Using flat fields from /data/2022-12/Luxi_173.h5
Crop data to the power of 2 sizes to work with 16bit precision, output size in x dimension 2048


queue size 000 |  |████████████████████████████████████████| 100.0% 


In [14]:
center_search = dxchange.read_tiff_stack(args.out_path_name+ 'recon_slice0425_center1734.16.tiff', in)

TypeError: read_tiff_stack() missing 1 required positional argument: 'ind'